In [1]:
!pip install selenium

import requests # 웹 페이지 소스를 얻기 위한 패키지(기본 내장 패키지이다.)
from bs4 import BeautifulSoup # 웹 페이지 소스를 얻기 위한 패키지, 더 간단히 얻을 수 있다는 장점이 있다고 한다.
from datetime import datetime                                # (!pip install beautifulsoup4 으로 다운받을 수 있다.)
import pandas as pd # 데이터를 처리하기 위한 가장 기본적인 패키지
import time # 사이트를 불러올 때, 작업 지연시간을 지정해주기 위한 패키지이다. (사이트가 늦게 켜지면 에러가 발생하기 때문)
import urllib.request #
from selenium.webdriver import Chrome
import json
import re     
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt



In [17]:
comment_data = pd.read_csv('문재인.csv')

In [18]:
comment_data

,Video ID,Title,Comment
0,_EdM8YXh9EQ,"(한밤토크) 박원순, 대사관 마지막 통화가 충격! 문재인 감빵 가자! / 신의한수 ...",지금에 문정권은 나라망치는 정권이다 ... 하루 빨리 퇴출 시키지 안으면 정말 ...
1,_EdM8YXh9EQ,"(한밤토크) 박원순, 대사관 마지막 통화가 충격! 문재인 감빵 가자! / 신의한수 ...",공범들이다!~★ 철저히조사해서 처벌하라!!
2,_EdM8YXh9EQ,"(한밤토크) 박원순, 대사관 마지막 통화가 충격! 문재인 감빵 가자! / 신의한수 ...",마지막 통화 내용이나\r\n말해주세요
3,_EdM8YXh9EQ,"(한밤토크) 박원순, 대사관 마지막 통화가 충격! 문재인 감빵 가자! / 신의한수 ...",신 대표님 마스크는 추천하지 마세요! 기능이 떨어지는 저급을 많은돈 지급하고 사게...
4,_EdM8YXh9EQ,"(한밤토크) 박원순, 대사관 마지막 통화가 충격! 문재인 감빵 가자! / 신의한수 ...",여가부가무엇하는곳인지알고싶어요
...,...,...,...
19341,bF2v8vbAG1I,[풀영상] 한국판 뉴딜 국민보고대회 - 문재인 대통령 기조연설 &quot;160조 ...,문재인 대통령님 얼마나 힘드 십니까?\r\n적극적으로 지원하고 응원합니다.\r\n그...
19342,bF2v8vbAG1I,[풀영상] 한국판 뉴딜 국민보고대회 - 문재인 대통령 기조연설 &quot;160조 ...,역시 우리 대통령이십니다.
19343,bF2v8vbAG1I,[풀영상] 한국판 뉴딜 국민보고대회 - 문재인 대통령 기조연설 &quot;160조 ...,확실히 연설은 존나 못한다
19344,bF2v8vbAG1I,[풀영상] 한국판 뉴딜 국민보고대회 - 문재인 대통령 기조연설 &quot;160조 ...,항상 응원합니다


In [19]:

#이모티콘 제거
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

#분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~,".\n\r#\ufeff\u200d]')

In [20]:
comment_list = []
for i in range(len(comment_data)):
    comment_list.append(comment_data['Comment'].iloc[i])

In [21]:
comment_result = []

for i in comment_list:
    tokens = re.sub(emoji_pattern,"",i)
    tokens = re.sub(han,"",tokens)
    comment_result.append(tokens)

comment_result = pd.DataFrame(comment_result, columns=["Comment"])

In [22]:
comment_result

,Comment
0,지금에 문정권은 나라망치는 정권이다 하루 빨리 퇴출 시키지 안으면 정말 큰 ...
1,공범들이다★ 철저히조사해서 처벌하라
2,마지막 통화 내용이나말해주세요
3,신 대표님 마스크는 추천하지 마세요 기능이 떨어지는 저급을 많은돈 지급하고 사게되...
4,여가부가무엇하는곳인지알고싶어요
...,...
19341,문재인 대통령님 얼마나 힘드 십니까적극적으로 지원하고 응원합니다그리고 감사합니다
19342,역시 우리 대통령이십니다
19343,확실히 연설은 존나 못한다
19344,항상 응원합니다


## 형태소 분석기를 통한 명사 추출¶

In [23]:
# !pip install konlpy

In [24]:
#한글 형태소 분석기인 konlpy사용 
#이때 konlpy에는 hannanum, kommoran, kkma, twitter이 있는데 해당 분석에서는 kkma사용
from konlpy.tag import Kkma
from konlpy.tag import Mecab
from konlpy.tag import Twitter
#apply lambda사용

In [25]:
from konlpy.tag import Twitter

def get_noun(comment_txt):
    twitter = Twitter()
    noun = []
    
    if len(comment_txt)>0:
        tw = twitter.pos(comment_txt)
        for i,j in tw:
            if j == 'Noun':
                noun.append(i)
    return noun
 
comment_result['token'] = comment_result['Comment'].apply(lambda x: get_noun(x))

C:\Users\user\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [26]:
comment_result['token']

0         [지금, 정권, 나라, 정권, 하루, 퇴, 정말, 난리, 확실, 요즘, 젊은이, 무얼]
1                                            [공범, 조사해, 처벌]
2                                         [마지막, 통화, 내용, 말]
3        [신, 대표, 마스크, 추천, 기능, 저급, 돈, 지급, 면, 신의, 한수, 정도,...
4                                             [여가부, 무엇, 곳]
                               ...                        
19341                       [문재인, 대통령, 얼마나, 십, 적극, 지원, 응원]
19342                                     [역시, 우리, 대통령, 니]
19343                                             [연설, 존나]
19344                                             [항상, 응원]
19345                                                   []
Name: token, Length: 19346, dtype: object